# Rule-Based System for Eligibility and Risk Assessment

This notebook implements a rule-based system using the CLIPS library to assess eligibility and risk based on patient data (part of the BMIN 5200 Final Project).

In [ ]:
## 20241211, BMIN5200 Final Project, clipspy demo (the Figure 6 in the report)
## Generaly adjust the code from our Homework 3

import clips
import sys
sys.path.append('./src/')
from clips_util import print_facts, print_rules, print_templates, build_read_assert

# Initialize the CLIPS environment
env = clips.Environment()

# Define facts and rules based on the new flowchart
rules = """
(defrule not-eligible
    (initial-facts (age ?age) (icu-stay ?icu_stay) (hours-in-icu ?hours))
    (or (and (< ?age 65) (neq ?icu_stay first))
        (< ?hours 24))
    =>
    (assert (eligibility not-eligible))
)

(defrule possibly-eligible
    (initial-facts (age ?age) (icu-stay ?icu_stay) (hours-in-icu ?hours))
    (and (>= ?age 65) (neq ?icu_stay first) (>= ?hours 24))
    =>
    (assert (eligibility possibly-eligible))
)

(defrule high-risk-pre-icu
    (eligibility possibly-eligible)
    (pre-icu-score ?score&:(> ?score 0.1))
    =>
    (assert (risk high-risk))
)

(defrule possibly-high-risk
    (eligibility possibly-eligible)
    (pre-icu-score ?score&:(< ?score 0.1))
    =>
    (assert (risk possibly-high-risk))
)

(defrule high-risk-severity
    (risk possibly-high-risk)
    (or (> ?sofa 5) (> ?saps 30) (> ?apache 17))
    =>
    (assert (risk high-risk))
)

(defrule predict-severity
    (risk possibly-high-risk)
    (and (<= ?sofa 5) (<= ?saps 30) (<= ?apache 17))
    =>
    (assert (action predict-severity))
)
"""

# Load rules into the environment
env.build(rules)

# Example facts assertion based on input
facts = [
    "(initial-facts (age 70) (icu-stay first) (hours-in-icu 48))",
    "(pre-icu-score 0.08)",
    "(sofa 4)",
    "(saps 28)",
    "(apache 16)"
]

for fact in facts:
    env.assert_string(fact)

# Run the environment to execute rules

env.reset();
env.run()

# Print the results
for fact in env.facts():
    print(fact)
